In [107]:
import os
import random
import pandas as pd
import numpy as np
from datetime import timedelta
import pandapower as pp
import pandapower.networks as pn
import pandapower.plotting as plot
from pandapower.timeseries import DFData
from pandapower.timeseries import OutputWriter
from pandapower.timeseries.run_time_series import run_timeseries
from pandapower.control import ConstControl
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
matplotlib.rcParams["timezone"] = "Europe/Berlin"

import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning) # pd.concat() sucks

import data_process
import importlib
importlib.reload(data_process_base)

try:
    import seaborn
    colors = seaborn.color_palette()
except:
    colors = ["b", "g", "r", "c", "y"]

In [108]:
# create random time series from import data
da = data_process.DataAction()
nc = data_process.net_calc()

da.folder_path = r"G:\\My Drive\\docs\\Education\\University\\ALUF\\SSE\\6 sem\\Bachelor Thesis\\data\\"
keep_cols = ['DE_KN_residential1_grid_import',
             'DE_KN_residential2_grid_import',
             'utc_timestamp']
da.imp_procc("house_data.csv", keep_cols)

Number of data frame segments =  90
dfList created successfully.


In [109]:
def night_rand():
    """Create time-limited randomly sampled series"""

    # choose random list number for df
    df_rand = np.random.choice(len(da.dfList[:-1]))  # last list dropped, likely not full

    # parse random data and convert to time index, limit to single series
    rand_col = np.random.randint(0,2)  # rand pick between two load profiles (in imported data)
    ts1 = da.parse_procc(da.dfList[df_rand].iloc[:,rand_col]) 

    # limit to night time, random date
    date1 = np.random.choice(da.unique_date(ts1)[1:-2])
    night1 = da.get_night(ts1, date1).copy()

    return night1


In [111]:
def load_sgen_make(load_number=5):
    """Create time series test dataframe for any number of loads.
    load_number goes from 1 to n, number designation of load in network"""

    # targets
    night_loads = pd.DataFrame()
    list_loads = []
    night_sgens = pd.DataFrame()
    list_sgens = []

    # generate list of Series for concatenation
    for i in range(1, load_number+1):
        night1 = night_rand()
        list_loads.append(pd.Series(night1.values, name='loadh_' + str(i)))
        list_sgens.append(pd.Series([0]*night1.shape[0], name='sgen_' + str(i)))

    # merge series into data frames
    night_loads = pd.concat(list_loads, axis=1) * da.conv_fac / 1000000 # to MW
    night_loads.index = night1.index
    night_sgens = pd.concat(list_sgens, axis=1)
    night_sgens.index = night1.index

    return night_loads, night_sgens


night_loads, night_sgens=load_sgen_make(12)
night_sgens

,sgen_1,sgen_2,sgen_3,sgen_4,sgen_5,sgen_6,sgen_7,sgen_8,sgen_9,sgen_10,sgen_11,sgen_12
date_time,,,,,,,,,,,,
2016-05-18 20:00:00+02:00,0,0,0,0,0,0,0,0,0,0,0,0
2016-05-18 20:01:00+02:00,0,0,0,0,0,0,0,0,0,0,0,0
2016-05-18 20:02:00+02:00,0,0,0,0,0,0,0,0,0,0,0,0
2016-05-18 20:03:00+02:00,0,0,0,0,0,0,0,0,0,0,0,0
2016-05-18 20:04:00+02:00,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2016-05-19 05:56:00+02:00,0,0,0,0,0,0,0,0,0,0,0,0
2016-05-19 05:57:00+02:00,0,0,0,0,0,0,0,0,0,0,0,0
2016-05-19 05:58:00+02:00,0,0,0,0,0,0,0,0,0,0,0,0


In [112]:
def sgen_rand(night_sgens, sgen_val):
    """Write sgen vals over random time window for cols"""
    sgens = night_sgens.columns

    for name in sgens:
        # writes directly to night_mw
        start, end = da.time_wind(night_sgens, 60)
        da.sgen_write(night_sgens, start, end, name, sgen_val)

    return night_sgens


night_mw = sgen_rand(night_sgens, 0.010)
night_mw

,sgen_1,sgen_2,sgen_3,sgen_4,sgen_5,sgen_6,sgen_7,sgen_8,sgen_9,sgen_10,sgen_11,sgen_12
date_time,,,,,,,,,,,,
2016-05-18 20:00:00+02:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-05-18 20:01:00+02:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-05-18 20:02:00+02:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-05-18 20:03:00+02:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-05-18 20:04:00+02:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2016-05-19 05:56:00+02:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-05-19 05:57:00+02:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-05-19 05:58:00+02:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [113]:
def end_results(sgen_val, iter, line_outputs, sgen_names):
    """Final boss"""
    
    # create df to store output
    col_names = line_outputs
    end_vals = pd.DataFrame(columns=col_names)
    end_times = pd.DataFrame(dtype='str', columns=col_names)

    for i in range(0, iter):
        # iterative scramble
        night_merge, load_names, sgen_names = auto_merge(iter)

        # assign generator value at random times
        night_mw = sgen_rand_2(ts=night_merge, sgen_names=sgen_names, sgen_val=0.01)
        da.night_mw = night_mw

        nc.euro_lv(night_mw)
        nc.four_loads_branched_out("res_line", "loading_percent")
        nc.four_loads_branched_run()
        nc.four_loads_branched_read_loadpct()

        nc.end_vals_step(nc.ll, end_vals)
        nc.end_times_step(nc.ll, end_times)

    return end_vals, end_times

night_merge, load_names, sgen_names = auto_merge()

iter = 1
sgen_val = 0.01 # typical val: 0.010 MW = 10kW
end_vals, end_times = end_results(sgen_val, iter, load_names, sgen_names)

ValueError: not enough values to unpack (expected 3, got 2)

In [ ]:
night_merge